**READ MJ2505 - OPTMIZATION_TUTORIAL_INSTRUCTIONS.PDF**

**Installing required packages**

In [1]:
pip install pulp

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


**Import the required libraries**

In [8]:
import pandas as pd
import pulp as pl
import numpy as np
import xlsxwriter as xl

**Import the data for the optimization from excel sheet**

*Price data in SEK/kWh*
*Demand in kWh*
*EV_ availability is binary value*

In [10]:
#input data reading from excel
days = 7 # number of days of simulation
worksheet = pd.read_excel('Input_data.xlsx')

#hourly electricity price
EV_available = worksheet.iloc[0:days*24, 1]
Demand = worksheet.iloc[0:days*24, 2]
Price_data = worksheet.iloc[0:days*24, 3]
SolarPV_generation = worksheet.iloc[0:days*24,4]

**Parameters for optimization**

In [11]:
Battery_capacity = 40 #Size of the EV battery in kWh
C_Rating = 1 #The rate at which the battery charges.
# The capacity of a battery is commonly rated at 1C, meaning that a fully charged battery rated at 1Ah should provide 1A for one hour\n",
Ptrafo = 1200 #The capacity is in kW

CS_losses = 0.96 #charging losses
DS_losses = 1.041 #discharging losses


Start_storage = 0.2*Battery_capacity # Initial Battery Capacity to start with

#We have one EV charger installed in a house
Charger_Power = 7.2 #Rated EV charger in kW- each charge will be able to supply this capacity

**Defining sets**

In [12]:
# time series parameter
#This is important for assigning the variables that are used in the optimization. 
#In python, you have to initialize lists/tuples/dictionaries first if they are used at some later point in the code.

Price = {x:1 for x in range(25)}
Demand_hourly = {x:1 for x in range(25)}
EV_availability = {x:1 for x in range(25)}
SolarPV = {x:1 for x in range(25)}

#sets initialization
I = list(range(24))
J = list(range(25)) #We have this until 25 , one  extra hour so as to ensure that the the storage level at the end of the day, acts as start storage level for enxt day
print(I)
print(J)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


**Defining excel file for writing results**

In [13]:
workbook = xl.Workbook('Results_optimization.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write(0, 0, 'Days')
worksheet.write(0, 1, 'Hour')
worksheet.write(0, 2, 'Demand')
worksheet.write(0, 3, 'SolarPV')
worksheet.write(0, 4, 'EV Available')
worksheet.write(0, 5, 'Charging power') #Total charging in an hour
worksheet.write(0, 6, 'Storage Level') #Total storage level in the battery of EV
worksheet.write(0, 7, 'Purchase from the grid') #Total amount of electricity purchased from the grid
worksheet.write(0, 8, 'SolarPV_consumption') 
worksheet.write(0, 9, 'Price') 
worksheet.write(0, 10, 'Cost of supply from grid') #Total of price*purchased electricity from the grid in an hour

0

**Function to move from one day to another**

In [17]:
#Here we write a function which can be called repeatedly where needed in the code
#This function is meant to fetch the data for each 24 hours of the optimization window.
#We run the optimization for every 24 hours and then it runs for the whole week

def assign(d):
    for y in I:
        Price[y] = Price_data[d*24 + y]
        Demand_hourly[y] = Demand[d*24 + y]
        EV_availability[y] = EV_available[d*24 + y]
        SolarPV[y] = SolarPV_generation[d*24 + y]

**Function where the optimization algorithm with contraints, objective functions and writing results is defined**

In [18]:
# Optimization model- Here we define another function
#Global variables are those values which are constant and will be accessed by the function. Therefore, it is important to assign them at the beginning of function
#You can try by removing one of them and the function will start producing an error

def Optimize():
    global Start_storage #Initiating variable for starting the storage of charged units
    global Battery_capacity #This is parameter as defined in the beginning
    global Charger_Power # This is parameter defined above
    global Ptrafo
    

    model = pl.LpProblem("Cost_minimize_charging_problem", pl.LpMinimize) # Here we define if the problem is minimization or maximization

    # Decision variables- The ones which offer flexibility and decision making  capability to optimizer
    
    Grid_purchase = pl.LpVariable.dicts('Grid_purchase', I, cat='Continuous', lowBound=0) # Syntax to specify the characteriestics of the variable
    Storage_level = pl.LpVariable.dicts('Storage_level', J,cat='Continuous', lowBound=0.2*Battery_capacity, upBound=0.8*Battery_capacity)
    Charge_hourly = pl.LpVariable.dicts('Charge_hourly', I, cat='Continuous', lowBound=0) #Amount of charge every hour, CS
    EV_charging_power = pl.LpVariable.dicts('EV_charging_power', I,cat='Continuous', lowBound=0)
    
    # Constraints
    
    #Storage level constraint
    model += Storage_level[0] == Start_storage       #Start of level of storage
    
    #in every hour
    for i in I:
        
        model += Grid_purchase[i] - Charge_hourly[i] - Demand_hourly[i] + SolarPV[i] == 0 # Energy Balance Constraint
        model += Grid_purchase[i] <= Ptrafo # Grid limit constraint
        
        model += Charge_hourly[i] <= C_Rating*Charger_Power*EV_availability[i] # Maximum charging limit in an hour
        
        if EV_availability[i] == 1 and EV_availability[i+1] == 0: # If EV leaves the house, the storage level
            model += Storage_level[i+1] == 0.8*Battery_capacity
            
        if EV_availability[i] == 0: #If EV is not available for charging
            model += Storage_level[i+1] == 0.2* Battery_capacity   
            
        if EV_availability[i] == 1: #If EV is avialable for charging           
            model += Storage_level[i+1] == Storage_level[i] + Charge_hourly[i]*CS_losses
       
        
  # Objective Function
    model += pl.lpSum (Grid_purchase[i]*Price[i] for i in I) #Defining the objective function
    model.solve()
    
    Start_storage = Storage_level[24].varValue      #for the next day we have to store the storage level of last hour of previous day

   #Writing results 
    for y in I:
        
        worksheet.write(x*24+y+1, 0, x+1) # Print number of days in excel
        worksheet.write(x*24+y+1, 1, y) # Print hours
        worksheet.write(x*24+y+1, 2, Demand_hourly[y])
        worksheet.write(x*24+y+1, 3, SolarPV[y])
        worksheet.write(x*24+y+1, 4, EV_availability[y])
        worksheet.write(x*24+y+1, 5, Charge_hourly[y].varValue)
        worksheet.write(x*24+y+1, 6, Storage_level[y].varValue)
        worksheet.write(x*24+y+1, 7, Grid_purchase[y].varValue)
        #worksheet.write(x*24+y+1, 8, Demand_hourly[y].varValue-Grid_purchase[y].varValue)
        worksheet.write(x*24+y+1, 9, Price[y])
        worksheet.write(x*24+y+1, 10, Grid_purchase[y].varValue*Price[y])
            
  #Running the model code

    model.solve()
    #Printing some results, not necessary though
    print('Day:')
    print(x+1)
    print (pl.LpStatus[model.status])
    print (pl.value(model.objective))

**Run the model**

In [19]:
#Using the function optimize to run the model for assigned set of data, here in our case it is one week as provided in the input data sheet

for x in range(days):
    assign(x) #Calling the data assignmment function
    Optimize()# Calling the optimize function
workbook.close()


Day:
1
Optimal
3.4135777116000003
Day:
2
Optimal
3.4659500972000004
Day:
3
Optimal
2.5087045708
Day:
4
Optimal
2.77092342515
Day:
5
Optimal
2.5670029479000003
Day:
6
Optimal
2.4011329717000005
Day:
7
Optimal
1.91931091725
